## Q1. Getting the embeddings model

In [1]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')

/Users/hani/Documents/llm-zoomcamp/llm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
user_question = "I just discovered the course. Can I still join it?"
vector = embedding_model.encode(user_question)
print("The first value of the embeded question: ", round(vector[0], 3))

The first value of the embeded question:  0.078


## Prepare the documents

In [3]:
# Load the documents with ids that we prepared in the module:

import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [4]:
ml_documents = [doc for doc in documents if doc['course'] == 'machine-learning-zoomcamp']
len(ml_documents)

375

In [5]:
ml_documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

## Q2. Creating the embeddings

In [6]:
# we will create an embedding for both question and answer fields.
# We want to put all of them into a single matrix `X`:

embeddings = []

for doc in ml_documents:
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'
    embeddings.append(embedding_model.encode(qa_text))

import numpy as np
X = np.array(embeddings)
X.shape

(375, 768)

In [7]:
vector.shape

(768,)

In [9]:
a = vector.dot(vector)
a.shape

()

## Q3. Search

In [10]:
# The vectors returned from the embedding model are already normalized 
# (you can check it by computing a dot product of a vector with itself - it should return something very close to 1.0)

print(vector.dot(vector))

# let's compute the cosine similarity between the vector from Q1 
# and the matrix from Q2. 

scores = X.dot(vector)
print("Highest Score of Cosine Similarity Scores: ", round(scores.max(), 3))

0.9999999
Highest Score of Cosine Similarity Scores:  0.651


## Vector search

In [11]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=ml_documents, embeddings=X)
search_engine_result = search_engine.search(vector, num_results=5)
search_engine_result[0]

{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
 'section': 'General course-related questions',
 'question': 'The course has already started. Can I still join it?',
 'course': 'machine-learning-zoomcamp',
 'id': 'ee58a693'}

## Q4. Hit-rate for our search engine
We will use the hitrate metric for evaluation.

In [12]:
# load the ground truth dataset
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [45]:
from tqdm.auto import tqdm
def hit_rate_cal(ground_truth, search_engine, num_results=5):
    cnt = 0
    for q in tqdm(ground_truth):
        query = q['question']
        doc_id_true = q['document']

        vector_query = embedding_model.encode(query)
        search_engine_result = search_engine(vector_query, num_results=num_results)

        if any(result['id'] == doc_id_true for result in search_engine_result):
            cnt += 1

    return cnt/len(ground_truth)

hit_rate = hit_rate_cal(ground_truth, search_engine.search, num_results=5)
print("Q4, Hit-rate for our search engine: ", round(hit_rate, 3))

100%|██████████| 1830/1830 [00:45<00:00, 40.31it/s]

Q4, Hit-rate for our search engine:  0.94


## Q5. Indexing with Elasticsearch

In [35]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 
es_client.info()

ObjectApiResponse({'name': 'e24d72127ec7', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'JtVSBZLTR4O14TjZqdG1Pw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [36]:
# Create the index with the same settings as in the module (but change the dimensions)

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [39]:
# Index the embeddings (note: you've already computed them)

for doc, embedding in zip(ml_documents, X):
    doc['question_text_vector'] = embedding

In [40]:
for doc in tqdm(ml_documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 375/375 [00:01<00:00, 242.26it/s]


In [41]:
def elastic_search_knn(vector, num_results):
    knn = {
        "field": "question_text_vector",
        "query_vector": vector,
        "k": num_results,
        "num_candidates": 10000,
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query,
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])
        result_docs[-1]['score'] = hit['_score']


    return result_docs

In [42]:
result = elastic_search_knn(vector, num_results=5)
result

[{'question': 'The course has already started. Can I still join it?',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693',
  'score': 0.82532895},
 {'question': 'I just joined. What should I do next? How can I access course materials?',
  'section': 'General course-related questions',
  'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to D

In [43]:
print("Q5, ID of the document with the highest score: ", result[0]['id'])

Q5, ID of the document with the highest score:  ee58a693


## Q6. Hit-rate for Elasticsearch

In [46]:
hit_rate_es = hit_rate_cal(ground_truth, elastic_search_knn, num_results=5)
print("Q6, Hit-rate for our Elastic Search: ", round(hit_rate_es, 3))

100%|██████████| 1830/1830 [00:35<00:00, 51.84it/s]

Q6, Hit-rate for our Elastic Search:  0.94
